In [1]:
import random

import cv2
import numpy as np

Download images from directory.

In [2]:
from image import read_image, show_image

In [3]:
images, image_names = read_image("InputImages")

In [5]:
image = images[0]

In [6]:
show_image("OutputImage", image)

# Stratification

Get lines from the image.

In [7]:
from line import get_lines, draw_line

In [8]:
lines = get_lines(image)
len(lines)

40

Plot lines.

In [10]:
ouput_image = image.copy()
# Getting the lines form the image
for line in lines:
    r = random.randint(0, 256)
    g = random.randint(0, 256)
    b = random.randint(0, 256)
    draw_line(ouput_image, line, (r, g, b))
show_image("OutputImage", ouput_image)

Cluster lines to find vanishing points.

In [11]:
from clustering import cluster_lines

In [12]:
n_clusters = 3

In [13]:
clusters = cluster_lines(lines, n_clusters=n_clusters)
clusters

defaultdict(list,
            {0: [array([[443.        ,   0.75049156]], dtype=float32),
              array([[261.       ,   0.6981317]], dtype=float32),
              array([[245.       ,   0.6806784]], dtype=float32),
              array([[569.       ,   0.7853982]], dtype=float32),
              array([[361.      ,   0.715585]], dtype=float32),
              array([[2.3900000e+02, 1.7453292e-02]], dtype=float32),
              array([[260.       ,   0.6806784]], dtype=float32),
              array([[363.       ,   0.7330383]], dtype=float32),
              array([[2.4700000e+02, 1.7453292e-02]], dtype=float32),
              array([[306.      ,   0.715585]], dtype=float32),
              array([[585.       ,   0.7853982]], dtype=float32),
              array([[532.       ,   0.7853982]], dtype=float32),
              array([[303.       ,   0.6981317]], dtype=float32),
              array([[2.4100000e+02, 1.7453292e-02]], dtype=float32),
              array([[254.       ,   0.698131

Show image with clustered lines and find vanishing points.

In [18]:
from homography import get_vanishing_point, get_nearest_points

In [15]:
vanishing_points = [get_vanishing_point(clusters[c]) for c in clusters]
vanishing_points

[array([-5415.984 , -1955.7689], dtype=float32),
 array([-1668.4064,  8764.042 ], dtype=float32),
 array([-1087.7777,  -159.3853], dtype=float32)]

In [16]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(n_clusters)]

output_image = image.copy()
for c in clusters:
    vp = vanishing_points[c]
    for i, line in enumerate(clusters[c]):
        draw_line(output_image, line, colors[c])
    cv2.circle(output_image, (int(vp[0]), int(vp[1])), 10, colors[c], -1)

show_image("OutputImage", output_image) 

In [19]:
two_points, _ = get_nearest_points(vanishing_points)
two_points

[array([-5415.984 , -1955.7689], dtype=float32),
 array([-1087.7777,  -159.3853], dtype=float32)]

In [90]:
point = [*max(two_points, key = lambda x: np.linalg.norm(x)), 1]
point

[-5415.984, -1955.7689, 1]

In [91]:
target_point = [0, 1, 0]

In [92]:
import math

In [125]:
def rotate_along_x(point: np.ndarray, target_point: np.ndarray) -> np.ndarray:
    theta_x = math.atan2(target_point[2], target_point[1]) - math.atan2(point[2], point[1])
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(theta_x), -np.sin(theta_x)],
        [0, np.sin(theta_x), np.cos(theta_x)],
    ])
    
    return R_x

In [126]:
def rotate_along_z(point: np.ndarray, target_point: np.ndarray) -> np.ndarray:
    theta_z = math.atan2(target_point[1], target_point[0]) - math.atan2(point[1], point[0])
    R_z = np.array([
        [np.cos(theta_z), -np.sin(theta_z), 0],
        [np.sin(theta_z), np.cos(theta_z), 0],
        [0, 0, 1],
    ])
    
    return R_z

In [128]:
R_z = rotate_along_z(point, target_point)
next_point = R_z @ point
next_point

array([1.11022302e-16, 1.41421356e+00, 1.00000000e+00])

In [129]:
R_x = rotate_along_x(next_point, target_point)
next_point = R_x @ next_point
next_point

array([1.11022302e-16, 1.73205081e+00, 1.11022302e-16])

In [167]:
M = [
    [1, 0, 0],
    [0, 1 / next_point[1], 0],
    [0, 0, 1],
]

In [132]:
M @ next_point

array([1.11022302e-16, 1.00000000e+00, 1.11022302e-16])

In [168]:
H = M @ R_x @ R_z
H

array([[ 0.70710678, -0.70710678,  0.        ],
       [ 0.33333333,  0.33333333,  0.33333333],
       [-0.40824829, -0.40824829,  0.81649658]])

In [169]:
output_image = cv2.warpPerspective(image, H, (image.shape[1], image.shape[0]))

In [170]:
show_image("OutputImage", output_image)